This script contains utility code to clean up certain things

In [ ]:
# Cleans up spectro dataset - removes all .spec files that does not contain any valid data
from remucs.util import SpectrogramCollection
import os
from tqdm.auto import tqdm
from scripts.calculate import DATASET_PATH
from remucs.dataset import get_valid_bar_numbers

deletable = []
for file in tqdm(os.listdir(DATASET_PATH), desc="Checking files..."):
    if file.endswith(".spec.zip"):
        try:
            spec = SpectrogramCollection.load(os.path.join(DATASET_PATH, file))
            valids = get_valid_bar_numbers(spec)
            if len(valids) == 0:
                deletable.append(file)
        except Exception as e:
            print(f"Error loading {file}: {e}")
            deletable.append(file)

# Removes the files
for line in deletable:
    os.remove(os.path.join(DATASET_PATH, line.strip()))